In [97]:
from polars import read_csv, col, Int64, min_horizontal

# Load data

In [67]:
df = read_csv("ML_TAKES_ENCODED.csv")

# Generate Features

## Strike Zone

### Turn Strikes and Balls from categorical to numerical feature

In [69]:
df = df.with_columns((col("PITCHCALL") == "StrikeCalled").cast(Int64).alias("IS_STRIKE"))

### Strike Zone Features

#### Determine if in zone

In [80]:
df = df.with_columns(
    ((col("BOT_ZONE") <= col("PLATELOCHEIGHT")) 
     & 
     (col("PLATELOCHEIGHT") <= col("TOP_ZONE"))).cast(Int64).alias("IN_ZONE"))

#### Determine how close to the edge of the zone the ball was

In [81]:
df = df.with_columns(
    min_horizontal(
        (col("PLATELOCHEIGHT") - col("BOT_ZONE")).abs(),
        (col("PLATELOCHEIGHT") - col("TOP_ZONE")).abs()
    ).alias("NEAR_VERT_EDGE")
)

In [82]:
PLATE_CENTER_WIDTH_FT = 0.708

In [83]:
df = df.with_columns((col("PLATELOCSIDE") - PLATE_CENTER_WIDTH_FT).abs().alias("NEAR_HORZ_EDGE"))

In [84]:
df = df.with_columns(PLATE_LOC_SIDE=col("PLATELOCSIDE"))
df = df.with_columns(PLATE_LOC_HEIGHT=col("PLATELOCHEIGHT"))

### Pitch Movement Features

#### Approach Angle

In [85]:
df = df.with_columns(VERT_APPROACH=col("VERTAPPRANGLE"))
df = df.with_columns(HORZ_APPROACH=col("HORZAPPRANGLE"))

#### Spead and Break

TODO: Need to test if these features are relevant or just noise

In [86]:
df = df.with_columns(INDUCED_VERT_BREAK=col("INDUCEDVERTBREAK"))
df = df.with_columns(HORZ_BREAK=col("HORZBREAK"))
df = df.with_columns(REL_SPEED=col("RELSPEED"))

### Remove nulls from critical features

In [99]:
df = df.drop_nulls(subset=features + ["IS_STRIKE"])

NameError: name 'FEATURE_COLS' is not defined

# Final Features

In [93]:
features = [
    "PLATE_LOC_HEIGHT",
    "PLATE_LOC_SIDE",
    "TOP_ZONE",
    "BOT_ZONE",
    "IN_ZONE",
    "NEAR_VERT_EDGE",
    "NEAR_HORZ_EDGE",
    "BALLS",
    "STRIKES",
    "VERT_APPROACH",
    "HORZ_APPROACH"
]

# Train Model

## Generate X and Y

In [94]:
x = df[features]

In [90]:
y = df["IS_STRIKE"]